In [1]:
    import collections, abc, io, IPython, functools

    if __name__ == '__main__': 
        get_ipython = IPython.get_ipython
    __all__ = "Jay J Json D Dict L List S String Context Schema Patch T Toml".split()

In [2]:
    get_data = lambda x: get_data(x.data) if hasattr(x, 'data') else x

In [3]:
    list_types = list, collections.UserList
    str_types = str, collections.UserString
    dict_types = dict, collections.UserDict

In [4]:
    class Stype(abc.ABCMeta):
        """Base class for short shortened string formatters."""
        def __mod__(Stype, str): return Jay.load(Stype.load(str))
        format = __call__ = __mod__

In [5]:
    class Jay(metaclass=Stype):
        def load(object):
            object = get_data(object)
            if isinstance(object, (dict, collections.UserDict)): return D(object)
            if isinstance(object, (list, collections.UserList)): return L(object)        
            if isinstance(object, (str, collections.UserString)): return S(object)  
            return object
    J = Jay

In [6]:
    class Json(metaclass=Stype): 
        """>>> Json%'{"a": "b"}'
    {'a': 'b'}
    """
        load = __import__('json').loads

In [7]:
    class Csv(metaclass=Stype): 
        """>>> Csv('''a, b, c
    ... 1, 2, 3''')
    [['a', ' b', ' c'], ['1', ' 2', ' 3']]
    """
        def load(object, *, delimiter=','):
            import csv
            if isinstance(object, tuple):
                delimiter, object = object 
            return L(list(csv.reader(io.StringIO(object), delimiter=delimiter)))

In [8]:
    def cfg(str):
        parser = __import__('configparser').ConfigParser()
        parser.read_string(str)
        return {section: dict(parser[section]) for section in parser.sections()}

    class Cfg(metaclass=Stype): 
        """>>> Cfg%'[header]\\na: b'
    {'header': {'a': 'b'}}
    """
        load = cfg

In [9]:
    class Object:
        def __getitem__(O, object):
            return Jay(__import__('jsonpointer').resolve_pointer(O.data, object))        
                
        def __add__(O, object, *, op='add'):
            if not isinstance(object, Patch):
                object = Patch([
                    {'op': op, 'path': key, **({} if op == 'replace' else {'value': value} )}
                    for key, value in (object.items() if isinstance(object, dict) else zip(object, ['']*len(object)))
                ])
            return object(O.data)
        
        __floordiv__ = functools.partialmethod(__add__, op='replace')
        
        __sub__ = functools.partialmethod(__add__, op='remove')
        
        def __or__(O, object):
            return __import__('jsondiff').diff(O.data, object)
        
        def __and__(O, object):
            O.update({'@context': object})
            return O
        
        __truediv__ = __floordiv__

        
        def __matmul__(O, object):
            return Context(object) @ O.data
    
        def __mod__(O, object):
            return Context(object) % O.data
                
        def __call__(O, object):
            return Schema(object)(O.data)
        
        def __setitem__(J, path, value):
            if path.startswith('/'):
                return J + {path: value}
            return super().__setitem__(path, value)
        
            
        def __wrapped__(O): raise AttributeError('__wrapped__')
            
        def __pos__(O): 
            data = dict(O.data)
            return Context(data.pop('@context')) @ data     
        
        def __neg__(O): 
            data = dict(O.data)
            return Context(data.pop('@context')) % data
        
    O = Object

In [10]:
    class String(O, collections.UserString):
        """>>> s = S("foo")
    >>> type(s) # doctest: +ELLIPSIS
    <...S...>

    >>> s["/1"]
    'o'
    
    """
    S = String

In [11]:
    class List(O, collections.UserList): 
        """>>> l = L(['a', 'foo'])
    >>> type(l) # doctest: +ELLIPSIS
    <...L...>
    >>> l['/0']
    'a'
    >>> l['/1/1']
    'o'
    >>> l + {'/2': 3}
    ['a', 'foo', 3]

    >>> L([1,2]) | [1]
    {delete: [1]}
    >>> L([1,2])-{'/1'}
    [1]
    """
    L = List

In [12]:
    class Patch(__import__('jsonpatch').JsonPatch, L): 
        """>>> l = L(['a', 'foo'])
    >>> type(l) # doctest: +ELLIPSIS
    <...L...>
    >>> l['/0']
    'a'
    >>> l['/1/1']
    'o'
    """
        def __call__(Patch, object):
            return Jay(Patch.apply(object))
        @property
        def data(Patch): return Patch.patch

In [13]:
    class Dict(O, collections.UserDict):
        """>>> d = Json%'{"a": [1, {"b": ["foo", 3, "bar"]}]}'
    >>> d
    {'a': [1, {'b': ['foo', 3, 'bar']}]}
    >>> d += {'/c': ['baz', None]}
    >>> d
    {'a': [1, {'b': ['foo', 3, 'bar']}], 'c': ['baz', None]}
    >>> d - {'/a', '/c/1'}
    {'c': ['baz']}
    >>> (d + {'/g': 'testing'}) @ {'g': 'https://foo'}
    [{'https://foo': [{'@value': 'testing'}]}]
    >>> d = (d + {'/g': 'testing'}) & {'g': 'https://foo'}
    >>> +d
    [{'https://foo': [{'@value': 'testing'}]}]
    """
        def __deepcopy__(D, state):
            return type(D)(D.data)
        
    D = Dict

In [14]:
    class Schema(D):
        """>>> Schema({'type': 'string'})('asdf')
    'asdf'
    
    >>> with __import__('pytest').raises(__import__('jsonschema').ValidationError): 
    ...     Schema({'type': 'string'})(1)
    """
        def __call__(Schema, object):
            __import__('jsonschema').validate(object, Schema.data)
            return object

In [15]:
    class Context(D):
        """>>> context = Context({"gh": "https://github.com"})
    >>> context@'gh'
    'https://github.com'
    >>> context@{'gh': 'deathbeds'}
    [{'https://github.com': [{'@value': 'deathbeds'}]}]
    """

        def __mod__(Context, object):
            return D(__import__('pyld').jsonld.compact(object, Context.data))
        
        
        def __matmul__(Context, object):
            if isinstance(object, str):                
                return Context.expand(object, Context.data)
            object.update({'@context': Context.data.get('@context', Context.data)})
            return L(__import__('pyld').jsonld.expand(object))

        @staticmethod
        def expand(str, ctx):
            """>>> Context.expand("gh", {"gh": "https://github.com"})
    'https://github.com'
    >>> Context.expand('q', {})
    'q'
    """
            object = __import__('pyld').jsonld.expand({str: '', '@context': ctx})
            if object: return next(iter(object[0]))
            return str

In [16]:
    class Yaml(metaclass=Stype): 
        """>>> Y%'[a, b]'
    ['a', 'b']
    >>> Y%'{a: b}'       
    {'a': 'b'}
    """
        def load(str): 
            import ruamel
            return ruamel.yaml.load(io.StringIO(str), Loader=ruamel.yaml.Loader)
            
    Y = Yaml

In [17]:
    class Toml(metaclass=Stype): 
        """>>> T%'title = "TOML Example"'
    {'title': 'TOML Example'}
    """
        def load(str): return __import__('toml').loads(str)
    T = Toml

In [18]:
    if __name__ == '__main__':
        import pidgin, nbconvert, black, IPython
        display = IPython.display.display
        with open('jason.py', 'w') as f:
            f.write(black.format_str(nbconvert.PythonExporter(config={
                'TemplateExporter': {'exclude_input_prompt': True}
            }, preprocessors=[pidgin.publishing.TanglePreProcessor()]).from_filename('jason.ipynb')[0], mode=black.FileMode()))
            if 0:
                with IPython.utils.capture.capture_output(stderr=False, stdout=False):
                    !pyreverse --show-builtin  --module-names=y -osvg jason 
            display(IPython.display.SVG('classes.svg'))
            with IPython.utils.capture.capture_output():
                !isort jason.py
        if 10:
            !pyflakes jason.py
        __import__('doctest').testmod()